In [1]:
from pathlib import Path

import requests

Path management

In [2]:
!pwd

/home/jobquiroz/full_stack_deep_learning/lab01/notebooks


In [3]:
# Fixing path
import os

os.getcwd()   # Verify where it is right now...

'/home/jobquiroz/full_stack_deep_learning/lab01/notebooks'

In [4]:
# Solution, go to lab directory:
os.chdir('/home/jobquiroz/full_stack_deep_learning/lab01/')

Download MNIST dataset

In [5]:
def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"
    
    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename

data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

Decompressing, deserialization and reading.

In [6]:
import gzip
import pickle


def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)

In [7]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

From arrays to tensors

In [8]:
import torch


x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

Tensors are defined by their contents: they are big rectangular blocks of numbers.

In [9]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


Indexing

In [10]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [11]:
#Num dimensions
x_train.ndim, y_train.ndim

(2, 1)

In [12]:
x_train[0, 0], y_train[0]

(tensor(0.), tensor(5))

In [13]:
# Shapes
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


### Random handwritten digits


In [15]:
# re-execute this cell for more samples
import random

import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes

import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(*metadata.DIMS)).image  # the image itself

tensor(8)


### From scratch model

In [16]:
import math
import torch


weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [17]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [18]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [19]:
bs = 64  # batch size

xb = x_train[0:bs]  # a batch of inputs
outs = model(xb)  # outputs on that batch

print(outs[0], outs.shape)  # outputs on the first element of the batch

tensor([-2.5404, -2.6016, -2.5248, -2.4114, -2.5581, -2.2841, -1.8198, -2.2158,
        -2.1511, -2.2032], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [20]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [21]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.1406)


In [22]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [23]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [24]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3945, grad_fn=<NegBackward0>) tensor(2.3026)


In [25]:
loss = loss_func(outs, yb)

loss.backward()

In [26]:
bias.grad

tensor([-0.0381, -0.0422,  0.0153, -0.0230, -0.0165,  0.0292,  0.0389,  0.0122,
         0.0483, -0.0241])

In [27]:
lr = 0.5  # learning rate hyperparameter
epochs = 2  # how many epochs to train for

for epoch in range(epochs):  # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):  # in batches of size bs, so roughly n / bs of them
        start_idx = ii * bs  # we are ii batches in, each of size bs
        end_idx = start_idx + bs  # and we want the next bs entires

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad():  # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # ACHTUNG: PyTorch doesn't assume you're done with gradients
            #          until you say so -- by explicitly "deleting" them,
            #          i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()

In [28]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0820, grad_fn=<NegBackward0>) tensor(1.)


In [29]:
# re-execute this cell for more samples
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(0)


### torch.nn components

In [30]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [31]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))  # should be unchanged from above!

tensor(0.0820, grad_fn=<NllLossBackward0>) tensor(1.)


Classy

In [33]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()  # the nn.Module.__init__ method does import setup, so this is mandatory
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [34]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()  # instantiated as an object
print(model(xb)[:4])  # callable like a function
loss = loss_func(model(xb), yb)  # composable like a function
loss.backward()  # we can still take gradients through it
print(model.weights.grad[::17,::2])  # and they show up in the .grad attribute

tensor([[-0.1158, -0.4842,  0.1280, -0.4428, -0.0124,  0.2508, -0.3444, -0.3650,
          0.0300, -0.0837],
        [-0.3843, -0.0834,  0.0929, -0.4112,  0.5065,  0.7480, -0.4524, -1.1187,
         -0.3566, -0.3108],
        [ 0.0712, -0.6250,  0.1582, -0.2846, -0.1893, -0.1965, -0.0759, -0.2788,
          0.1486, -0.4321],
        [ 0.4173, -0.4888,  0.3655,  0.0792, -0.0726,  0.1883,  0.0698, -0.9419,
         -0.2531, -0.3400]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0052,  0.0111,  0.0120,  0.0045,  0.0068],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0342,  0.0067,  0.0665,  0.041

In [35]:

print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[-0.0177, -0.0369, -0.0078,  ..., -0.0306,  0.0027,  0.0455],
        [ 0.0113, -0.0143,  0.0618,  ...,  0.0315, -0.0494, -0.0522],
        [ 0.0348,  0.0339, -0.0554,  ..., -0.0474, -0.0362,  0.0511],
        ...,
        [ 0.0383,  0.0019, -0.0314,  ...,  0.0505, -0.0211, -0.0347],
        [ 0.0088, -0.0135, -0.0186,  ...,  0.0070,  0.0372, -0.0421],
        [-0.0112, -0.0248,  0.0078,  ..., -0.0134,  0.1023,  0.0508]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [36]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs
            end_idx = start_idx + bs
            xb = x_train[start_idx:end_idx]
            yb = y_train[start_idx:end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():  # finds params automatically
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [37]:
print(accuracy(model(xb), yb))

tensor(1.)


## More refactoring

In [38]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [39]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)  # pytorch finds the nn.Parameters inside this nn.Module

    def forward(self, xb):
        return self.lin(xb)  # call nn.Linear.forward here

In [40]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))  # loss is still close to 2.3

tensor(2.2877, grad_fn=<NllLossBackward0>)


In [41]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [43]:
list(model.children())

[Linear(in_features=784, out_features=10, bias=True)]

In [44]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0194,  0.0059,  0.0076,  ...,  0.0052, -0.0344, -0.0054],
        [-0.0152,  0.0074,  0.0338,  ..., -0.0222, -0.0286,  0.0147],
        [ 0.0098,  0.0208,  0.0176,  ..., -0.0285, -0.0227,  0.0185],
        ...,
        [-0.0013,  0.0180, -0.0030,  ..., -0.0355,  0.0187,  0.0113],
        [-0.0308, -0.0090, -0.0044,  ..., -0.0256, -0.0192,  0.0015],
        [ 0.0255,  0.0167, -0.0049,  ..., -0.0022,  0.0129, -0.0018]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0136,  0.0260,  0.0141,  0.0105,  0.0312, -0.0033, -0.0323, -0.0283,
         0.0302, -0.0045], requires_grad=True)


In [48]:
list(model.parameters())[0]

Parameter containing:
tensor([[ 0.0194,  0.0059,  0.0076,  ...,  0.0052, -0.0344, -0.0054],
        [-0.0152,  0.0074,  0.0338,  ..., -0.0222, -0.0286,  0.0147],
        [ 0.0098,  0.0208,  0.0176,  ..., -0.0285, -0.0227,  0.0185],
        ...,
        [-0.0013,  0.0180, -0.0030,  ..., -0.0355,  0.0187,  0.0113],
        [-0.0308, -0.0090, -0.0044,  ..., -0.0256, -0.0192,  0.0015],
        [ 0.0255,  0.0167, -0.0049,  ..., -0.0022,  0.0129, -0.0018]],
       requires_grad=True)

### Optimizer

In [50]:
from torch import optim

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [51]:
model = MNISTLogistic()
opt = configure_optimizer(model)

print("before training:", loss_func(model(xb), yb), sep="\n\t")

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training:", loss_func(model(xb), yb), sep="\n\t")

before training:
	tensor(2.3263, grad_fn=<NllLossBackward0>)
after training:
	tensor(0.8633, grad_fn=<NllLossBackward0>)


### Dataset

In [52]:
from text_recognizer.data.util import BaseDataset


train_ds = BaseDataset(x_train, y_train)

ModuleNotFoundError: No module named 'text_recognizer.data'